# Lecture 31: Simulated Annealing

```{note}
...
```
---

## Overview

Expanding over the Threshold Accepting algorithm, the Simulated Annealing algorithm probabilistically accepts worse solutions to escape local optima traps in search of high-quality solutions. Specifically, the algorithm takes inspiration from the annealing process in metallurgy, wherein a material is first heated and then slowly cooled down, reaching a stable state. Analogous to this process, the Simulated Annealing algorithm initialises the search at a sufficiently high temperature – $T_0$, and then cools down the search to temperature T_k in k^th iteration of the algorithm, based on the cooling schedule – $T$. These cooling schedules can include linear cooling – $T_k=T_0-θk$, exponential cooling – $T_k=T_0 θ^k$, logarithmic cooling – $T_k=T_0/log⁡(k+1)$, and even adaptive cooling, wherein the temperature increases/decreases dynamically based on search progress. Hence, in the Simulated Annealing algorithm, it is the cooling schedule that is responsible for managing exploration and exploitation of the search space. Specifically, a slow cooling schedule will result in high exploration but low exploitation, while a fast cooling schedule will render low exploration but high exploitation. It is thus pertinent to select an appropriate cooling schedule that fits the nature of the problem and associated solution landscape.

The Simulated Annealing algorithm begins with an initial solution – $s_o$, which serves as the current solution – $s$, as well as the best solution – $s^*$. Hereafter, in each iteration the algorithm applies a defined set of transformations to the current solution defined by the neighbourhood – $N(s)$, to generate a new solution – $s'$. The algorithm accepts this new solution as the current solution based on an acceptance probability – $p$, which is a function of the objective function evaluation of the current solution – $f(s)$ and the new solution – $f(s)$. Specifically, the algorithm accepts a better new solution with probability $p=1$, and a worse new solution with a probability determined by the Boltzmann function – $p=\text{exp}⁡((f(s)-f(s'))/T_k)$. In addition, if this new solution is better than the best the solution, then the algorithm updates the best solution as well. Finally, in each iteration, the algorithm updates the temperature based on the cooling schedule. The algorithm iterates until it reaches a threshold level of solution quality or computation effort, returning the best solution in the end. 

---

## Pseudo Code

1. **Procedure** $\text{SA}(s_o, (N, T))$
2. $s ← s_o$ &emsp;<small>// initialize current solution $s$ as the initial solution $s_o$</small>
3. $s^* ← s$ &emsp;<small>// initialize best solution $s^*$ as the current solution</small>
4. $k ← 0$ &emsp;<small>// initialize iteration counter $k$ at zero</small>
5. $T ← T_k$ &emsp;<small>// initialize current temperature based on the cooling schedule $T$</small>
6. **while** $!\text{converged}$ **do** &emsp;<small>// repeat until converged</small>
7. &emsp;$s' {R \atop ←} N(s)$ &emsp;<small>// generate new random solution from neighborhood $N$ of the current solution</small>
8. &emsp;$\Lambda \sim U(0, 1)$ &emsp;<small>// generate a random number $\Lambda$ uniformly from $[0, 1]$</small>
9. &emsp;$\lambda {R \atop ←} \Lambda$ &emsp;<small>// draw a random value $\lambda$ from $\Lambda$</small>
10. &emsp;**if** $f(s') < f(s)$ **then** &emsp;<small>// if the new solution is better than the current solution</small>
11. &emsp;&emsp;$s ← s'$ &emsp;<small>// update the current solution to the new solution</small>
12. &emsp;**else if** $\lambda < \exp((f(s) - f(s')) / T)$ **then** &emsp;<small>// else accept the new solution with a small probability</small>
13. &emsp;&emsp;$s ← s'$ &emsp;<small>// update the current solution to the new solution</small>
14. &emsp;**end if**
15. &emsp;**if** $f(s) < f(s^*)$ **then** &emsp;<small>// if the current solution is better than the best solution</small>
16. &emsp;&emsp;$s^* ← s$ &emsp;<small>// update the best solution to the current solution</small>
17. &emsp;**end if**
18. &emsp;$k ← k + 1$ &emsp;<small>// update iteration counter</small>
19. &emsp;$T ← T_k$ &emsp;<small>// update current temperature based on the cooling schedule</small>
20. **end while**
21. **return** $s^*$ &emsp;<small>// return the best solution</small>

---

## Implementation

---

## Case Study

---

```{note}
...
```